In [1]:
library(arcgisbinding)
arc.check_product()

Warning message:
"package 'arcgisbinding' was built under R version 3.5.0"*** Please call arc.check_product() to define a desktop license.


product: ArcGIS Pro ( 12.2.0.12813 )
license: Advanced 
version: 1.0.1.230

### Raster Data

The R-ArcGIS bridge recently expanded its capabilities to include working with raster data. This means that the **arcgisbinding** package has been updated to include some new functions specific for manipulating and converting ArcGIS raster layers into R recognized raster objects (from the R package **raster**). If you have an older version of the bridge, you will need to update your **arcgisbinding** package in order to access this functionality. For information on how to do so, see the section on upgrading the **arcgisbinding** package in the Jupyter Notebook R-bridge-install-and-setup. 

#### arc.raster Functionality

While the `arc.open()` function works with all ArcGIS data types, whether vector or raster, to establish a connection to your desired ArcGIS data, the **arcgisbinding** package has seperate functions for converting and manipulating data depending on the type of data you wish to bring into R. The `arc.raster()` function is what is used for converting and manipulating a raster layer contained within any type of ArcGIS supported format before finishing the conversion process into R using the `as.raster()` function. 

To begin, let's start with the simplest use case. We simply wish to convert an ArcGIS raster layer into an R recognized raster object. In this case, after using the `arc.open()` function to establish a connection with our desired data set, we can use the `arc.raster()` function on the result generated from `arc.open()`, followed by the `as.raster()` function, to convert our data into an R raster object. If we are working with a single band, the result will be a _RasterLayer_ object. If we are working with multiple bands, the results will be a _RasterBrick_ object. Let's take our `World_Distance_to_Waster_Stud_SM` file geodatabase raster as an example:

```R
distance_to_water_ras <- as.raster(arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM")))
```

Our `World_Distance_to_Waster_Stud_SM` file geodatabase raster is now in R as a _RasterLayer_ and any R function that takes a **raster** package object as part of its arguments can now be used on it. To check this, we can run the following line in R and check the class of the resulting object. 


In [11]:
distance_to_water_ras <- as.raster(arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM")))
class(distance_to_water_ras)
str(distance_to_water_ras)

[1] "RasterLayer"
attr(,"package")
[1] "raster"

Formal class 'RasterLayer' [package "raster"] with 12 slots
  ..@ file    :Formal class '.RasterFile' [package "raster"] with 13 slots
  .. .. ..@ name        : chr ""
  .. .. ..@ datanotation: chr "INT1U"
  .. .. ..@ byteorder   : chr "little"
  .. .. ..@ nodatavalue : num NA
  .. .. ..@ NAchanged   : logi FALSE
  .. .. ..@ nbands      : int 1
  .. .. ..@ bandorder   : chr "BIL"
  .. .. ..@ offset      : int 0
  .. .. ..@ toptobottom : logi TRUE
  .. .. ..@ blockrows   : int 0
  .. .. ..@ blockcols   : int 0
  .. .. ..@ driver      : chr ""
  .. .. ..@ open        : logi FALSE
  ..@ data    :Formal class '.SingleLayerData' [package "raster"] with 13 slots
  .. .. ..@ values    : int [1:30250] 15 14 14 13 11 9 8 7 5 4 ...
  .. .. ..@ offset    : num 0
  .. .. ..@ gain      : num 1
  .. .. ..@ inmemory  : logi TRUE
  .. .. ..@ fromdisk  : logi FALSE
  .. .. ..@ isfactor  : logi FALSE
  .. .. ..@ attributes: list()
  .. .. ..@ haveminmax: logi TRUE
  .. .. ..@ min       : int 0
  .. .. .

In addition to this simplest use case shown above, the `arc.raster()` function also offers advancted manipulations for raster layers. Next, we will cover some of the additional options that exist. 

> Note: The parameter options covered here are only some of the options that exist within the `arc.raster()` function. For full options and details, please refer to the documentation on this function. 

In [ ]:
?arc.raster()

##### Pixel Selections

If you only wish to bring in a subset of pixels from your raster, you can define them using the `nrow` and `ncol` parameters of the `arc.raster()` function. This can be useful if you wish to test out R functionality on a smaller subset of your raster first before bringing in the full raster layer into R. 

To use, simply specify the number of rows and columns you wish to bring into R. 

```R
distance_to_water_ras <- as.raster(arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM"), nrow = 100, ncol = 100))
```

In [30]:
distance_to_water_ras <- as.raster(arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM"), nrow = 100, ncol = 100))
dim(distance_to_water_ras)

[1] 100 100   1

##### Extent 

You can adjust the extent for the raster object you bring into R by using the `extent` parameter and specifying your desired xmax, ymax, xmin, and ymin values in a numeric vector. Order does matter here, so be mindful as you select your values. 

In the example below, we are adjusting the extent of our raster to have a xmax value of 31, a ymax value of 25.5, a xmin value of 30, and a ymin value of 24.5. 

```R
distance_to_water_ras_extent <- as.raster(arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM"), nrow = 100, ncol = 100, extent = c(31, 25.5, 30, 24.5)))

```

We can test out the results in the R console below. 

In [15]:
#Original extent values
distance_to_water_ras <- as.raster(arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM"), nrow = 100, ncol = 100))
distance_to_water_ras@extent

#Modified extent values
distance_to_water_ras_extent <- as.raster(arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM"), nrow = 100, ncol = 100, extent = c(31, 25.5, 30, 24.5)))
distance_to_water_ras_extent@extent

class       : Extent 
xmin        : 30.98066 
xmax        : 32.96726 
ymin        : -25.46767 
ymax        : -24.50616 

class       : Extent 
xmin        : 30 
xmax        : 31 
ymin        : 24.5 
ymax        : 25.5 

##### Resampling

If you elect to create a subset of your original data, you might wish to resample. In which case, the `arc.select()` function offers options for resampling your data through the `resample_type` argument. 

Here, we are creating a subset of our original data by specifying a desired number of rows and columns and then resampling the subset using the Cubic Convolution method. 

```R
distance_to_water_ras <- as.raster(arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM"), nrow = 100, ncol = 100))
```

We can explore this result in the R console below:

In [29]:
#Original Pixel Values
distance_to_water_ras <- as.raster(arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM"), nrow = 100, ncol = 100))
head(distance_to_water_ras, 15)

#Resampled Pixel Values
distance_to_water_ras_resamp <- as.raster(arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM"), nrow = 100, ncol = 100, resample_type="CubicConvolution"))
head(distance_to_water_ras, 15)

[1] 14 13  8  5  4  6 10 14 18 22 27 30 33 35 39

[1] 14 13  8  5  4  6 10 14 18 22 27 30 33 35 39

##### Reprojecting

Similarly to the `arc.select()` function, we can reproject our data by using the `sr` parameter and specifying either the ESPG reference number, the WKT, or proj4 associcated with the ESPG reference number for the desired projection. Our `World_Distance_to_Waster_Stud_SM` file geodatabase raster is currently not projected. If we wished to add a projection, we could do so in the following way:

ESPG reference number

```R
distance_to_water_ras <- arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM"), nrow = 100, ncol = 100, sr = 102003)
```
By checking the value of the `sr` attribute value, we can see our spatial reference has now changed.

In [49]:
#Original Spatial Reference Information
distance_to_water_ras <- arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM"), nrow = 100, ncol = 100)
distance_to_water_ras$sr

#ESPG reference number
distance_to_water_ras <- arc.raster(arc.open("data/data.gdb/World_Distance_to_Water_Stud_SM"), nrow = 100, ncol = 100, sr = 102003)
distance_to_water_ras$sr


$WKT
[1] "GEOGCS[\"WGS 84\",DATUM[\"World Geodetic System 1984\",SPHEROID[\"WGS 84\",6378137.0,298.257223563]],PRIMEM[\"Greenwich\",0.0],UNIT[\"degree\",0.0174532925199433]]"

$WKID
[1] 0

$WKT
[1] "PROJCS[\"USA_Contiguous_Albers_Equal_Area_Conic\",GEOGCS[\"GCS_North_American_1983\",DATUM[\"D_North_American_1983\",SPHEROID[\"GRS_1980\",6378137.0,298.257222101]],PRIMEM[\"Greenwich\",0.0],UNIT[\"Degree\",0.0174532925199433]],PROJECTION[\"Albers\"],PARAMETER[\"False_Easting\",0.0],PARAMETER[\"False_Northing\",0.0],PARAMETER[\"Central_Meridian\",-96.0],PARAMETER[\"Standard_Parallel_1\",29.5],PARAMETER[\"Standard_Parallel_2\",45.5],PARAMETER[\"Latitude_Of_Origin\",37.5],UNIT[\"Meter\",1.0]]"

$WKID
[1] 102003

#### as.raster Functionality

We have already seen the `as.raster()` function in some of the examples above, but to properly define, the `as.raster()` function is from the R package **raster** and is designed to help create raster objects. It is utilized in the **arcgisbinding** package to help finish the conversion process from an ArcGIS raster type to an R **raster** package object. Full documentation on the function can be found [here](https://www.rdocumentation.org/packages/raster/versions/2.6-7/topics/as.raster). 

#### Mosaic Datasets

So far, we have covered the **arcgisbinding** package functionality pertaining to most raster types, except for Mosaic rasters. The bridge handles Mosaic data sets in a slight different way when bringing them based on their structure. If you wish to work with a Mosaic data set in R, you will do the following to bring it into R from ArcGIS:

First, you will use the `arc.open()` function just like you would for any other raster layer. Second, when you use the `arc.raster()` function, you will specify the method, regarding the family of sorting rules used by ArcGIS, as well as, the operator, used to resolve overlaps in ArcGIS. This will enable the bridge to properly read in your Mosaic data set. 

Example:

```R
```



In [64]:
mosaic_ras <- arc.raster(arc.open("data/data.gdb/bullseye_5bandMD"), mosaic = list(method = "NorthWest" , operator = "First"))


ERROR: Error in .call_proxy("dataset.open", .Object, path): cannot open dataset


In [61]:
list(method = "North-West" , operator = "First")


$method
[1] "North-West"

$operator
[1] "First"

#### Practice Session: Raster Manipulations